In [2]:
import torch 
import math as math
import torch.nn as nn
class inputembedding(nn.Module):
    def __init__(self,d_model:int,vocab_size:int):
        super().__init__()
        self.vocab_size = vocab_size  #length of each vector 
        self.embedding = nn.embedding(d_model,vocab_size)
        
    def forward(self,x):
        return self.embedding(x)* math.sqrt(d_model) # tensor of size and dimension


In [3]:
class positional_embedding(nn.Module):
    
    def __init__(self, d_model: int, seq_len: int, dropout: float):
        
        super().__init__()
        self.d_model = d_model 
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
    
    
# create a matrix of shape
        pe = torch.zeros(seq_len,d_model)
# create vector of shapes 
        po = torch.arrange(0,seq_len,dtype=torch.float).unsequence(1)
        
        # cos 
        
        div_term = torch.exp(torch.arranage(0,d_model,2,dtype=float))*(-math.log(10000.0)/d_model)
        
        #apply sin 
        
        pe[:,0::2]= torch.sin(po*div_term)
        pe[:,1::2] = torch.cos(po*div_term)
        
        pe= pe.unsqueeze(0)
        self.buffers('pe',pe)
        
        def forward(self,x):
            x = x+(self.pe[:, :x.shape(1),:]).requires_grad_(False)
            return self.dropout(x)

In [4]:
class layernorm(nn.Module):
    def __init__(self,eps: float = 10**-6):
        super().__init__()
        self.eps = eps
        self.alpha = nn.parameter(torch.ones(1)) #multiply 
        self.bias = nn.parameter(torch.zeros(1)) 
        
        
    def forward(self,x):
        
        mean = x.mean(dim=-1,keepdim=True)
        std = x.std(dim=-1,keepdim=True)
        return self.alpha* (x-mean)/(std+eps)+self.bias
        
        
        

In [5]:
class feedforward(nn.Module):
    
    def __init__ (self,d_model:int,d_diff:int,dropout: float):
        super.__init__ ()
        self.linear1 = nn.linear(d_model,d_diff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.linear(d_model,d_diff)
        
        def forward():
            x = self.linear1(x)
            x = self.relu(x)
            x= self.dropout(x)
            x = self.linear2(x)
            return x

In [ ]:
class multiheadattention(nn.Module):
    
    def __init__(self, d_model:int, num_heads:int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        assert d_model % num_heads == 0 , "d_model must be divisible by num_heads"
        self.d_k = d_model // num_heads # dk is dimension of each head
        self.w_q = nn.Linear(d_model,d_model)
        self.w_k = nn.Linear(d_model,d_model)
        self.w_v = nn.Linear(d_model,d_model)
        self.w_o = nn.Linear(d_model,d_model)
        self.dropout = nn.Dropout(dropout)
        
        @staticmethod
        
        def attention(query,key,value,mask=None,dropout=None):
            d_k = query.shape[-1]
            attention_scores = (query.matmul(key.transpose(-2,-1)))/math.sqrt(d_k)
            if mask is not None:
                attention_scores = attention_scores.masked_fill(mask==0,-1e9)
                attention_score  = attention_scores.softmax(dim=-1) # batch size x num_heads x seq len x seq len
                if dropout is not None:
                    attention_score = dropout(attention_score)
                    
                return attention_score.matmul(value), attention_score
        
        
        def forward(self,query,key,value,mask=None):
            query = self.w_q(query) # batch size x seq len x d_model
            key = self.w_k(key)
            value = self.w_v(value)
            # (bathch size x seq len x num_heads x d_k ) -- ( batch size x num_heads x seq len x d_k)     
            query =query.view(query.size[0], query.size[1], self.num_heads, self.d_k).transpose(1,2)
            key = key.view(key.size[0], key.size[1], self.num_heads, self.d_k).transpose(1,2)
            value = value.view(value.size[0], value.size[1], self.num_heads, self.d_k).transpose(1,2)
            
        x,self.attention_score = multiheadattention.attention(query,key,value,mask=mask,dropout=self.dropout)
        # batch size x num_heads x seq len x d_k -- batch size x seq len x num_heads x d_k --- batch size x seq len x d_model
        x = x.transpose(1,2).contiguous().view(x.size[0],-1,self.num_heads*self.d_k)
        # batch size x seq len x d_model - batch size x seq len x d_model
        return self.w_o(x)
    

In [9]:
class residual(nn.Module):
    def __init__ (self, dropout : float):
        super(). __init__ ()
        self.dropout = nn.Dropout(dropout)
        self.norm = layernorm(d_model)
        
    def forward(self,x,sublayer):
        return x+ self.dropout(sublayer(self.norm(x)))
        

In [ ]:
class decoderblock(nn.Module):
    def __init__(self,self_attenion: multiheadattention, cross_attention: multiheadattention,feedforward: feedforward, residual: residual, d_model:int, num_heads:int, d_diff:int, dropout: float):
        super().__init__()
        self.self_attention = self_attenion
        self.cross_attention = cross_attention
        self.feedforward = feedforward
        self.residual = nn.ModuleList([residual(dropout) for _ in range(3)])
        
    def forward(self,x,encoder_output,src_mask,tgt_mask): # soource mask is coming from encoder and target mask is coming from decoder
        x = self.residual[0](x, lambda x: self.self_attention(x,x,x,tgt_mask))
        x = self.residual[1](x, lambda x: self.cross_attention(x,encoder_output,encoder_output,src_mask))
        x = self.residual[2](x, self.feedforward)
        return x
    
class decoder(nn.Module):
    def __init__ (self,layers: nn.ModuleList, norm: layernorm):
        super().__init__()
        self.layers = layers
        self.norm = norm
        
    def forward(self,x,encoder_output,src_mask,tgt_mask):
        for layer in self.layers:
            x = layer(x,encoder_output,src_mask,tgt_mask)
        return self.norm(x)

In [13]:
class projection(nn.Module):
    def __init__ (self,d_model:int,vocab_size:int):
        super().__init__()
        self.linear = nn.Linear(d_model,vocab_size)
        
    def forward(self,x):
        return torch.log_softmax(self.linear(x),dim=-1)
    
class transformer(nn.Module):
    def __init__ (self, encoder:encoder ,decoder:decoder,src_embedding: inputembedding, tgt_embedding: inputembedding,src_positional_embedding: positional_embedding, tgt_positional_embedding: positional_embedding, projection: projection):
        super().__init__()
        self.encoder = encoder 
        self.decoder = decoder 
        self.src_embedding = src_embedding
        self.tgt_embedding = tgt_embedding
        self.src_positional_embedding = src_positional_embedding
        self.tgt_positional_embedding = tgt_positional_embedding
        self.projection = projection
        
    def encode(self,src,src_mask):
        src = self.src_embedding(src)
        src = self.src_positional_embedding(src)
        return self.encoder(src,src_mask)
    
    def decode(self,encoder_output,src_mask,tgt,tgt_mask):
        tgt = self.tgt_embedding(tgt)
        tgt = self.tgt_positional_embedding(tgt)
        return self.decoder(tgt,encoder_output,src_mask,tgt_mask)
    
    
    def project(self,x):
        return self.projection(x):
    
    
    def build_transformers(src_vocab_size:int, tgt_vocab_size:int, seq_len:int, d_model:int=512, num_heads:int=8, d_diff:int=2048, num_layers:int=6, dropout: float=0.1):
        
        # create the embedding layers 
        src_embedding = inputembedding(d_model,src_vocab_size)
        tgt_embedding = inputembedding(d_model,tgt_vocab_size)
        
        
        src_positional_embedding = positional_embedding(d_model,seq_len,dropout)
        tgt_positional_embedding = positional_embedding(d_model,seq_len,dropout)
        
        # create the an encoder blocks
        encoder_block = []
        for _ in range(num_layers):
            e_self_attention = multiheadattention(d_model,num_heads,dropout)
            e_feedforward_network = feedforward(d_model,d_diff,dropout)
            e_residual_connection = residual(dropout)
            encoder_layer = encoderblock(e_self_attention,e_feedforward_network,e_residual_connection,d_model,num_heads,d_diff,dropout)
            encoder_block.append(encoder_layer)
            
            
        # create the decoder blocks 
        decoder_block = []
        for _ in range(num_layers):
            d_self_attention = multiheadattention(d_model,num_heads,dropout)
            d_cross_attention = multiheadattention(d_model,num_heads,dropout)
            d_feedforward_network = feedforward(d_model,d_diff,dropout)
            d_residual_connection = residual(dropout)
            decoder_layer = decoderblock(d_self_attention,d_cross_attention,d_feedforward_network,d_residual_connection,d_model,num_heads,d_diff,dropout)
            decoder_block.append(decoder_layer)
    # create encoder and decoder 
    encoder = encoder(nn.ModuleList(encoder_block),layernorm(d_model))
    decoder = decoder(nn.ModuleList(decoder_block),layernorm(d_model))
    # create projection layer
    projection_layer = projection(d_model,tgt_vocab_size)
    
    # create the transformer
    transformer_model = transformer(encoder,decoder,src_embedding,tgt_embedding,src_positional_embedding,tgt_positional_embedding,projection_layer)
    
    #initialize the parameters 
    
    for p in transformer_model.parameters():
        if p.dim()>1:
            nn.init.xavier_uniform_(p)
            
            
            return transformer_model

SyntaxError: invalid syntax (2297516494.py, line 32)